# Split PDF Book into Multiple Text File

In [1]:
import os
from PyPDF2 import PdfReader, PdfWriter

In [38]:
book = PdfReader('Mining Text Data.pdf')

In [121]:
# actual page number
actual_page = 12
chapter_first_page = [1,11,42,76,128,161,221,256,293,318,355,379,409,458,513]
chapter_first_page[0]+actual_page
page = book.pages[chapter_first_page[6]+actual_page]
print(page.extract_text())

Chapter 7
TRANSFER LEARNING FOR TEXT
MINING
Weike Pan
Hong Kong University of Science and Technology
Clearwater Bay, Kowloon, Hong Kong
weikep@cse.ust.hk
Erheng Zhong
Hong Kong University of Science and Technology
Clearwater Bay, Kowloon, Hong Kong
ezhong@cse.ust.hk
Qiang Yang
Hong Kong University of Science and Technology
Clearwater Bay, Kowloon, Hong Kong
qyang@cse.ust.hk
Abstract Over the years, transfer learning has received much attention in machine
learning research and practice. Researchers have found that a major
bottleneck associated with machine learning and text mining is the lack
of high-quality annotated examples to help train a model. In response,
transfer learning oﬀers an attractive solution for this problem. Various
transfer learning methods are designed to extract the useful knowledge
from diﬀerent but related auxiliary domains. In its connection to text
mining, transfer learning has found novel and useful applications. In
this chapter, we will review some most recent

- Chapter 1: 1-11
- Chapter 2: 11-42
- Chapter 3: 42-76
- Chapter 4: 76-128

In [129]:
book_page_index = [x+12 for x in chapter_first_page]
print(book_page_index)

[13, 23, 54, 88, 140, 173, 233, 268, 305, 330, 367, 391, 421, 470, 525]


In [140]:
for i in range(len(book_page_index)-1):
    print(f'starting at page {book_page_index[i]}, ending at page {book_page_index[i+1]}')
    start_page = book_page_index[i]
    end_page = book_page_index[i+1]
    num_pages = end_page - start_page + 1
    content = ''
    for j in range(num_pages):
        print(j+start_page)
        content += book.pages[j+start_page].extract_text()
    with open(f"doc_{i+1}.txt", "w", encoding='utf-8') as file:
        file.write(content)
        file.close()

starting at page 13, ending at page 23
13
14
15
16
17
18
19
20
21
22
23
starting at page 23, ending at page 54
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
starting at page 54, ending at page 88
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
starting at page 88, ending at page 140
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
starting at page 140, ending at page 173
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
starting at page 173, ending at page 233
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220

# Text Similarity (Chapter Similarity)

In [20]:
import gensim
import nltk
import re
import numpy as np
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.stem.porter import *
from gensim import corpora

In [7]:
corpus = PlaintextCorpusReader('data', '.+\.txt')
fids = corpus.fileids()
print(fids)

['doc_1.txt', 'doc_10.txt', 'doc_11.txt', 'doc_12.txt', 'doc_13.txt', 'doc_14.txt', 'doc_2.txt', 'doc_3.txt', 'doc_4.txt', 'doc_5.txt', 'doc_6.txt', 'doc_7.txt', 'doc_8.txt', 'doc_9.txt']


In [8]:
# Create a dictionary from book
book_docs = [corpus.words(f) for f in fids]
book_lower = [[w.lower() for w in doc] for doc in book_docs]
book_alpha = [[w for w in doc if re.search('^[a-z]+$',w)] for doc in book_lower]
stop_list = stopwords.words('english')            
book_stop = [[w for w in doc if w not in stop_list] for doc in book_alpha]
stemmer = PorterStemmer()
book_stem = [[stemmer.stem(w) for w in doc] for doc in book_stop]
book_dict = corpora.Dictionary(book_stem)
print(book_dict)

Dictionary(11947 unique tokens: ['abil', 'abl', 'abstract', 'access', 'account']...)


In [5]:
# Convert all doc to list of sparse vectors
book_vecs = [book_dict.doc2bow(doc) for doc in book_stem]

In [12]:
from gensim import similarities
# Build an index from a list of sparse vectors
index = similarities.SparseMatrixSimilarity(book_vecs,11947)

In [18]:
# e.g. find similar chapter for chapter 1
chapter_1 = book_vecs[0]
sims = index[chapter_1]
print(list(enumerate(sims)))

[(0, 0.9999979), (1, 0.55079097), (2, 0.53323555), (3, 0.63295704), (4, 0.37265885), (5, 0.5907426), (6, 0.4736471), (7, 0.43834755), (8, 0.5010129), (9, 0.42730898), (10, 0.60467124), (11, 0.5869448), (12, 0.48561707), (13, 0.640863)]


# Text Summarization

In [30]:
#open text file in read mode
text_file = open("data/doc_1.txt", "r", encoding='utf-8')
 
#read whole file to a string
data = text_file.read()
 
#close file
text_file.close()
 
print(data)

Chapter 1
AN INTRODUCTION TO TEXT MINING
Charu C. Aggarwal
IBM T. J. Watson Research Center
Yorktown Heights, NY
charu@us.ibm.com
ChengXiang Zhai
University of Illinois at Urbana-Champaign
Urbana, IL
czhai@cs.uiuc.edu
Abstract
The problem of text mining has gained increasing attention in recent
years because of the large amounts of text data, which are created in
a variety of social network, web, and other information-centric applica-
tions. Unstructureddataistheeasiestformofdatawhichcanbecreated
in any application scenario. As a result, there has been a tremendous
need to design methods and algorithms which can eﬀectively process a
wide variety of text applications. This book will provide an overview
of the diﬀerent methods and algorithms which are common in the text
domain, with a particular focus on mining methods.
1. Introduction
Datamining isaﬁeld whichhasseen rapid advancesin recentyears[8]
because of the immense advances in hardware and software technology
which has lead to the 

In [31]:
import gensim
from gensim import corpora

# Load the text to be summarized
text = data

# Tokenize the text into individual words
tokens = gensim.utils.simple_preprocess(text)

# Create a dictionary from the tokens
dictionary = corpora.Dictionary([tokens])

# Create a corpus from the dictionary and tokens
corpus = [dictionary.doc2bow(tokens)]

# Train the LDA model with the corpus
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, passes=10)

# Get the top sentences from the LDA model
top_sentences = sorted(lda_model.show_topic(0, topn=10), key=lambda x: x[1], reverse=True)

# Generate the summary from the top sentences
summary = ' '.join([sent[0] for sent in top_sentences])

# Print the summary
print(summary)

the of text in and to data mining is for
